In [5]:
### 获取表格 ###
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# 设置 Chrome 浏览器选项
chrome_options = Options()
chrome_options.add_argument("--headless")  # 无头模式
chrome_options.add_argument("--disable-gpu")  # 禁用 GPU 加速

# 启动 Selenium WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# 目标 URL
url = "https://gwykl.fujian.gov.cn/kl2025/signupcount"

# 打开页面
driver.get(url)

# 等待页面加载
time.sleep(3)  # 你可以根据需要调整等待时间，或者使用显式等待

# 获取页面的 HTML
soup = BeautifulSoup(driver.page_source, 'html.parser')

# 查找表格
table = soup.find('table')
if table:
    # 解析表头
    headers = [th.get_text(strip=True) for th in table.find_all('th')]

    # 解析表格的每一行
    rows = []
    for tr in table.find_all('tr')[1:]:  # 跳过表头
        cells = tr.find_all('td')
        if len(cells) > 0:  # 确保该行有数据
            row = [cell.get_text(strip=True) for cell in cells]
            rows.append(row)

    # 将数据转换为 Pandas DataFrame
    df = pd.DataFrame(rows, columns=headers)

    # 输出 DataFrame 到 Excel 文件
    excel_file = '福建省公务员考试录用表格.xlsx'
    df.to_excel(excel_file, index=False)

    print(f"数据已成功保存到 {excel_file}")
else:
    print("未找到表格。")

# 关闭浏览器
driver.quit()


数据已成功保存到 福建省公务员考试录用表格.xlsx


In [66]:
### 修改表格格式，删除括号内容 ###
import pandas as pd
import re

# 读取 Excel 文件
file_path = '福建省公务员考试录用表格.xlsx'
df = pd.read_excel(file_path)

# 假设我们要修改的是第一列
first_column = df.iloc[:, 0]
second_column = df.iloc[:, 1]

# 定义一个函数，处理文本：删除括号及其中内容并添加空格
def process_text(text):
    # 使用正则表达式删除括号及其中的内容
    text = re.sub(r'\(.*?\)', '', str(text))  # 删除中括号及其中的内容
    text = text.strip()  # 去掉前后空格
    return text

# 应用处理函数到第一列
df.iloc[:, 0] = first_column.apply(process_text)
df.iloc[:, 1] = second_column.apply(process_text)

# 将修改后的数据保存回新的 Excel 文件
output_file = '福建省公务员考试录用表格_已处理.xlsx'
df.to_excel(output_file, index=False)

print(f"处理后的数据已保存到 {output_file}")


处理后的数据已保存到 福建省公务员考试录用表格_处理后.xlsx


In [75]:
### 筛选合并 ###
import pandas as pd

# 加载两个 Excel 文件
df1 = pd.read_excel('福建省公务员考试录用表格_已处理.xlsx')  # 处理后的表格
df2 = pd.read_excel('岗位-可报考.xlsx')  # 目标表格

# 假设第一列和第二列分别为 '列1' 和 '列2'，并且最后三列有效数据为 '列-3', '列-2', '列-1'
# 根据实际列名调整
df1 = df1[['单位', '职位', '招考数','报名数', '审核通过', '未审核']]  # 从第一个表中提取需要的列
df2 = df2[['单位','职位','单位所属系统','单位所属地区','单位层级','工作地点','职级','试卷类型','招收人数','招考范围','出生年月要求','性别要求','政治面貌要求','文化程度要求','学位要求','工作经历要求','所需专业','职位简介','其他要求','备注','联系方式'
]]  

# 使用合并操作，按照列1和列2进行匹配
merged_df = pd.merge(df2, df1, how='left', on=['单位', '职位'])

# 将合并后的数据保存为新的 Excel 文件
merged_df.to_excel('省考岗位-可报考.xlsx', index=False)

print("数据已成功合并并保存为 '省考岗位-可报考.xlsx'")


数据已成功合并并保存为 '合并后的表格.xlsx'


In [45]:
### 爬取一个职业查询表格 ### 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# 设置 Chrome 浏览器选项
chrome_options = Options()
chrome_options.add_argument("--headless")  # 无头模式
chrome_options.add_argument("--disable-gpu")  # 禁用 GPU 加速

# 启动 Selenium WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# 目标 URL
url = "https://gwykl.fujian.gov.cn/kl2025/position"

# 打开页面
driver.get(url)

# 等待页面加载
time.sleep(3)  # 你可以根据需要调整等待时间，或者使用显式等待

# 获取页面的 HTML
soup = BeautifulSoup(driver.page_source, 'html.parser')

# 查找包含表格的span
span = soup.find("span", id="datalist")
if span:
    table = span.find("table")
    if table:
        print(table)
        # 解析表头（奇数位置的td作为列名）
        headers = []
        rows = []
        
        # 获取表头，奇数位置的 td 作为列头
        th_cells = table.find_all('tr')  # 获取所有 tr 标签
        print(tr)
        for tr in th_cells:
            cells = tr.find_all('td')  # 获取每行的 td 标签
            if len(cells) > 0:
                row = []
                # 假设奇数位置的 td 是表头
                for i in range(0, len(cells) - 1, 2):  # 获取奇数位置的 td
                    headers.append(cells[i].get_text(strip=True))  # 获取表头内容
                # 假设偶数位置的 td 是数据内容
                for i in range(1, len(cells), 2):  # 获取偶数位置的 td
                    row.append(cells[i].get_text(strip=True))  # 获取数据内容
                if row:
                    rows.extend(row)  # 将每行作为一个子列表添加到 rows 中

#         print(rows)
                    
        # 将数据转换为 Pandas DataFrame
#         df = pd.DataFrame(rows, columns=headers)

#         # 输出 DataFrame 到 Excel 文件
#         excel_file = '福建省公务员考试职位查询.xlsx'
#         df.to_excel(excel_file, index=False)

#         print(f"数据已成功保存到 {excel_file}")
#     else:
#         print("未找到表格。")
# else:
#     print("未找到包含表格的 span 标签。")

# 关闭浏览器
driver.quit()


<table class="table table-bordered table-condensed table-responsive tb03"><tbody><tr><td class="tdT">单位代码/名称</td><td class="tdC">00031 福建省残疾人联合会</td><td class="tdT">职位代码/名称</td><td class="tdC">01 办公室</td></tr><tr><td class="tdT">单位所属系统</td><td class="tdC">参照管理人民团体和群众团体</td><td class="tdT">单位所属地区</td><td class="tdC">省直   单位层级：省直</td></tr><tr><td class="tdT">工作地点</td><td class="tdC">省直</td><td class="tdT">职级</td><td class="tdC">一级主任科员及以下</td></tr><tr><td class="tdT">试卷类型</td><td class="tdC">A类（省市综合管理）</td><td class="tdT">招收人数</td><td class="tdC">1</td></tr><tr><td class="tdT">招考范围</td><td class="tdC">全国</td><td class="tdT">年龄要求</td><td class="tdC">35周岁以下</td></tr><tr><td class="tdT">性别要求</td><td class="tdC">不限</td><td class="tdT">民族要求</td><td class="tdC">不限</td></tr><tr><td class="tdT">政治面貌要求</td><td class="tdC">不限</td><td class="tdT">文化程度要求</td><td class="tdC">本科及以上</td></tr><tr><td class="tdT">学位要求</td><td class="tdC">学士或以上学位</td><td class="tdT">学历类别要求</td><td class="tdC">不限</td></tr><

In [59]:
### 爬取一页职业查询表格 ###
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# 设置 Chrome 浏览器选项
chrome_options = Options()
chrome_options.add_argument("--headless")  # 无头模式
chrome_options.add_argument("--disable-gpu")  # 禁用 GPU 加速

# 启动 Selenium WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# 目标 URL
url = "https://gwykl.fujian.gov.cn/kl2025/position"

# 打开页面
driver.get(url)

# 等待页面加载
time.sleep(3)  # 你可以根据需要调整等待时间，或者使用显式等待

# 获取页面的 HTML
soup = BeautifulSoup(driver.page_source, 'html.parser')

# 查找包含表格的span
span = soup.find("span", id="datalist")
if span:
    tables = span.find_all("table")
    rows_ = []
    # 解析表头（奇数位置的td作为列名）
    headers = []
    for table in tables:
        if table:
            rows = []
            # 获取表头，奇数位置的 td 作为列头
            th_cells = table.find_all('tr')  # 获取所有 tr 标签
            for tr in th_cells:
                cells = tr.find_all('td')  # 获取每行的 td 标签
                if len(cells) > 0:
                    row = []
                    # 假设奇数位置的 td 是表头 并且表头只要统计一次
                    if table == tables[0]:
                        for i in range(0, len(cells) - 1, 2):  # 获取奇数位置的 td
                            headers.append(cells[i].get_text(strip=True))  # 获取表头内容
                    # 假设偶数位置的 td 是数据内容
                    for i in range(1, len(cells), 2):  # 获取偶数位置的 td
                        row.append(cells[i].get_text(strip=True))  # 获取数据内容
                    if row:
                        rows.extend(row)  # 将每行作为一个子列表添加到 rows 中
        if len(rows) == 22:
            rows_.append(rows)
#     print(rows_)
#     print(headers)
                    
    # 将数据转换为 Pandas DataFrame
    df = pd.DataFrame(rows_, columns=headers)

    # 输出 DataFrame 到 Excel 文件
    excel_file = '福建省公务员考试职位查询.xlsx'
    df.to_excel(excel_file, index=False)

else:
    print("未找到包含表格的 span 标签。")

# 关闭浏览器
driver.quit()


In [65]:
### 爬取所有职业查询表格 ###
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# 设置 Chrome 浏览器选项
chrome_options = Options()
chrome_options.add_argument("--headless")  # 无头模式
chrome_options.add_argument("--disable-gpu")  # 禁用 GPU 加速

# 启动 Selenium WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# 目标 URL
url = "https://gwykl.fujian.gov.cn/kl2025/position"

# 打开页面
driver.get(url)

# 等待页面加载
time.sleep(3)  # 你可以根据需要调整等待时间，或者使用显式等待

# 保存所有行数据
rows_for_all = []

page = 1
MAX_PAGE = 412 # 写死了

while True:
    # 获取页面的 HTML
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 查找包含表格的span
    span = soup.find("span", id="datalist")
    if span:
        tables = span.find_all("table")
        rows_ = []
        # 解析表头（奇数位置的td作为列名）
        headers = []
        for table in tables:
            if table:
                rows = []
                # 获取表头，奇数位置的 td 作为列头
                th_cells = table.find_all('tr')  # 获取所有 tr 标签
                for tr in th_cells:
                    cells = tr.find_all('td')  # 获取每行的 td 标签
                    if len(cells) > 0:
                        row = []
                        # 假设奇数位置的 td 是表头 并且表头只要统计一次
                        if table == tables[0]:
                            for i in range(0, len(cells) - 1, 2):  # 获取奇数位置的 td
                                headers.append(cells[i].get_text(strip=True))  # 获取表头内容
                        # 假设偶数位置的 td 是数据内容
                        for i in range(1, len(cells), 2):  # 获取偶数位置的 td
                            row.append(cells[i].get_text(strip=True))  # 获取数据内容
                        if row:
                            rows.extend(row)  # 将每行作为一个子列表添加到 rows 中
            if len(rows) == 22:
                rows_.append(rows)
    rows_for_all.extend(rows_)
    try:
        page = page + 1
        next_button = driver.find_element(By.XPATH,f"//a[text()='{page}']")
        if page > MAX_PAGE:
            print("没有更多页面，爬取结束")
            break
        else:
            next_button.click()
            time.sleep(5)
    except Exception as e:
        print("无法爬取")
        break
                    
# 将数据转换为 Pandas DataFrame
df = pd.DataFrame(rows_for_all, columns=headers)

# 输出 DataFrame 到 Excel 文件
excel_file = '福建省公务员考试职位查询.xlsx'
df.to_excel(excel_file, index=False)



# 关闭浏览器
driver.quit()


KeyboardInterrupt: 